# Merge Accuracy analysis

These scripts are executed when tools disagree on the existence of conflicts in order to estimate merge accuracy. You can control the tools being compared by tweaking the `TOOL_A` and `TOOL_B` variables.

In [ ]:
from helpers.sqlite_helpers import sql_query_to_pd

(TOOL_A, TOOL_B) = ('jdime', 'last_merge')
# (TOOL_A, TOOL_B) = ('spork', 'mergiraf')

## Situation 1 - Tool A reports a conflict and B is syntactically equivalent to the merge (Added false positive from A)

In [ ]:
scenarios_with_distinct_results_query = f"""
  SELECT
    *
  FROM
    global_executions_per_commit_filtered gepc
  WHERE
    {TOOL_A}_result != {TOOL_B}_result
"""

def count_false_positives_for_tool(tool_a: str, tool_b: str) -> str:
  return f"""
    SELECT
      scenario_id,
      '{tool_a.upper()}' as tool
    FROM
      {tool_b}_executions_per_commit_filtered
    WHERE
      scenario_id IN (SELECT scenario_id FROM ({scenarios_with_distinct_results_query}))
      AND result = 'SUCCESS_WITHOUT_CONFLICTS'
      AND outputs_equivalent = 1
  """

# A row in this table means that tool X produced a false positive in the scenario
scenarios_in_which_one_tools_find_conflicts_and_another_fully_matches_query = f"""
  {count_false_positives_for_tool(TOOL_A, TOOL_B)}
  UNION
  {count_false_positives_for_tool(TOOL_B, TOOL_A)}
"""

scenarios_in_which_one_tools_find_conflicts_and_another_fully_matches = sql_query_to_pd(scenarios_in_which_one_tools_find_conflicts_and_another_fully_matches_query)
scenarios_in_which_one_tools_find_conflicts_and_another_fully_matches

## Situation 2 - Tool A reports a conflict, B is not syntactically equivalent to the merge, but builds and tests pass (Added false positive from A)

In [ ]:
def get_scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query(tool_a: str) -> str:
    return f"""
    SELECT
      scenario_id,
      '{tool_a.upper()}' as tool_that_will_be_tested
    FROM
      {tool_a}_executions_per_commit_filtered
    WHERE
      scenario_id IN (SELECT scenario_id FROM ({scenarios_with_distinct_results_query}))
      AND result = 'SUCCESS_WITHOUT_CONFLICTS'
      AND outputs_equivalent = 0
  """

scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query = f"""
  {get_scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query(TOOL_A)}
  UNION
  {get_scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query(TOOL_B)}
"""

scenarios_in_which_one_tool_find_conflicts_but_the_other_passes_tests_query = f"""
    SELECT
        CASE WHEN UPPER(snv.tool_that_will_be_tested) = UPPER('{TOOL_A}') THEN UPPER('{TOOL_B}') ELSE UPPER('{TOOL_A}') END as tool,
        tr.scenario_id
    FROM
        test_results as tr
    JOIN
        ({scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query}) as snv
    ON
        tr.scenario_id = snv.scenario_id AND UPPER(tr.tool) == UPPER(snv.tool_that_will_be_tested)
    WHERE
        status = 'success'
        AND UPPER(tr.tool) IN (UPPER('{TOOL_A}'), UPPER('{TOOL_B}'))
    ORDER BY
        tr.scenario_id
"""
scenarios_in_which_one_tool_find_conflicts_but_the_other_passes_tests = sql_query_to_pd(scenarios_in_which_one_tool_find_conflicts_but_the_other_passes_tests_query)
scenarios_in_which_one_tool_find_conflicts_but_the_other_passes_tests

## Situation 3 - Tool A reports a conflict, B is not syntactically equivalent to the merge, builds and tests fail (False negative from B)

In [ ]:
scenarios_in_which_one_tool_find_conflicts_but_the_other_fails_tests_query = f"""
    SELECT
        snv.tool_that_will_be_tested as tool,
        tr.scenario_id
    FROM
        test_results as tr
    JOIN
        ({scenarios_in_which_one_tools_find_conflicts_but_the_other_do_not_match_merge_query}) as snv
    ON
        tr.scenario_id = snv.scenario_id AND UPPER(tr.tool) == snv.tool_that_will_be_tested
    WHERE
        status = 'failure'
        AND UPPER(tr.tool) IN (UPPER('{TOOL_A}'), UPPER('{TOOL_B}'))
    ORDER BY
        tr.scenario_id
"""
scenarios_in_which_one_tool_find_conflicts_but_the_other_fails_tests = sql_query_to_pd(scenarios_in_which_one_tool_find_conflicts_but_the_other_fails_tests_query)
scenarios_in_which_one_tool_find_conflicts_but_the_other_fails_tests

# Visualization

In [ ]:
false_positives_query = f"""
SELECT
    "Situation 1 - False Positive" AS situation,
    tool,
    scenario_id
FROM
    ({scenarios_in_which_one_tools_find_conflicts_and_another_fully_matches_query})
UNION
SELECT
    "Situation 2 - False Positive" AS situation,
    tool,
    scenario_id
FROM
    ({scenarios_in_which_one_tool_find_conflicts_but_the_other_passes_tests_query})
"""

false_negatives_query = f"""
SELECT
    "Situation 3 - False Negative" AS situation,
    tool,
    scenario_id
FROM
    ({scenarios_in_which_one_tool_find_conflicts_but_the_other_fails_tests_query})
"""

resume_query = f"""
SELECT * FROM ({false_positives_query})
UNION
SELECT * FROM ({false_negatives_query})
"""

visualization_query = f"""
    SELECT situation, tool, count(scenario_id) FROM ({resume_query}) GROUP BY situation, tool
"""
sql_query_to_pd(visualization_query)